In [1]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import re
import sys
from typing import List
import json

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

# Update this path to wherever tensorboard is installed in your environment
os.environ['TENSORBOARD_BINARY'] = '/opt/conda/envs/py310/bin/tensorboard'


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /opt/conda did not contain libcudart.so as expected! Searching further paths...
  warn(msg)


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
BASE_MODEL = "decapoda-research/llama-13b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [44]:
CUTOFF_LEN = 512

In [9]:
jsons = []
with open("../data/psychology_data.jsonl", "r") as f:
    for line in f:
        jsons.append(json.loads(line))

In [51]:
def split_text_into_chunks(text):
    # Split the text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Check if adding the current sentence exceeds the maximum chunk length
        if len(current_chunk) + len(sentence) > CUTOFF_LEN:
            chunks.append(current_chunk.strip())  # Add the current chunk to the list of chunks
            current_chunk = ""  # Reset the current chunk
        
        current_chunk += " " + sentence + " "  # Add the sentence to the current chunk
    
    if current_chunk:
        chunks.append(current_chunk.strip())  # Add the remaining chunk to the list of chunks
    
    return chunks

prompts = []
for book in jsons:
    splitted_text = split_text_into_chunks(book['text'].strip())
    if len(splitted_text):
        for i, c in enumerate(splitted_text):
            prompts.append({
                'id': f"text_id{book['text_id']}_chunk_id{i}",
                'text': c
            })

with open("../data/psychology_prompts.json", "w") as f:
    json.dump(prompts, f)

In [52]:
data = load_dataset("json", data_files="../data/psychology_prompts.json")
data["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jupyter/.cache/huggingface/datasets/json/default-4ad2c1236d91ef23/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 2433
})

In [54]:
data['train']

Dataset({
    features: ['id', 'text'],
    num_rows: 2433
})

In [55]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(text):
    return tokenize(text['text'])

In [56]:
train_val = data["train"].train_test_split(
    test_size=500, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/1933 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [66]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
EPOCHS = 5
OUTPUT_DIR = "experiments"  # Path to where your training model will be saved

In [67]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/tmp/ipykernel_4652/2918270676.py:10 in <module>                                             │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/var/tmp/ipykernel_4652/2918270676.py'                     │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/peft/mapping.py:120 in get_peft_model         │
│                                                                                                  │
│   117 │   │   return PeftModel(model, peft_config)                                               │
│   118 │   if isinstance(peft_config, PromptLearningConfig):                                      │
│   119 │   │   peft_config = _prepare_prompt_learning_config(peft_config, model_config)           │
│ ❱ 120 │   return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](model, peft_config)     │
│   121                                                                                            │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/peft/peft_model.py:662 in __init__            │
│                                                                                                  │
│    659 │   """                                                                                   │
│    660 │                                                                                         │
│    661 │   def __init__(self, model, peft_config: PeftConfig, adapter_name="default"):           │
│ ❱  662 │   │   super().__init__(model, peft_config, adapter_name)                                │
│    663 │   │   self.base_model_prepare_inputs_for_generation = self.base_model.prepare_inputs_f  │
│    664 │                                                                                         │
│    665 │   def forward(                                                                          │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/peft/peft_model.py:99 in __init__             │
│                                                                                                  │
│     96 │   │   self.base_model_torch_dtype = getattr(model, "dtype", None)                       │
│     97 │   │   if not isinstance(peft_config, PromptLearningConfig):                             │
│     98 │   │   │   self.peft_config[adapter_name] = peft_config                                  │
│ ❱   99 │   │   │   self.base_model = PEFT_TYPE_TO_MODEL_MAPPING[peft_config.peft_type](          │
│    100 │   │   │   │   self.base_model, self.peft_config, adapter_name                           │
│    101 │   │   │   )                                                                             │
│    102 │   │   │   self.set_additional_trainable_modules(peft_config, adapter_name)              │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/peft/tuners/lora.py:154 in __init__           │
│                                                                                                  │
│   151 │   │   self.model = model                                                                 │
│   152 │   │   self.forward = self.model.forward                                                  │
│   153 │   │   self.peft_config = config                                                          │
│ ❱ 154 │   │   self.add_adapter(adapter_name, self.peft_config[adapter_name])                     │
│   155 │                                                    

In [ ]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=5,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=1,
    save_steps=1,
    output_dir=OUTPUT_DIR,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [63]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [64]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))
 
model = torch.compile(model)

In [65]:
trainer.train()

/opt/conda/envs/py310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


A: torch.Size([1344, 5120]), B: torch.Size([5120, 5120]), C: (1344, 5120); (lda, ldb, ldc): (c_int(43008), c_int(163840), c_int(43008)); (m, n, k): (c_int(1344), c_int(5120), c_int(5120))
cuBLAS API failed with status 15
error detected

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/tmp/ipykernel_4652/4032920361.py:1 in <module>                                              │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/var/tmp/ipykernel_4652/4032920361.py'                     │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:1664 in train         │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:1932 in               │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1929 │   │   │   │   │   with model.no_sync():                                                 │
│   1930 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1931 │   │   │   │   else:                                                                     │
│ ❱ 1932 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1933 │   │   │   │                                                                             │
│   1934 │   │   │   │   if (                                                                      │
│   1935 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:2727 in training_step │
│                                                                                                  │
│   2724 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2725 │   │                                                                                     │
│   2726 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2727 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2728 │   │                                                                                     │
│   2729 │   │   if self.args.n_gpu > 1:                                                           │
│   2730 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /opt/conda/envs/py310/lib/python3.10/site-packages/transformers/trainer.py:2759 in compute_loss  │
│                                                                                                  │
│   2756 │   │   │   labels = inputs.pop("labels")                                                 │
│   2757 │   │   else:                                       

In [14]:
model.save_pretrained(OUTPUT_DIR)

In [17]:
from huggingface_hub import notebook_login
 
notebook_login()

In [18]:
model.push_to_hub("kmnis/DocScribe", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/kmnis/medVicuna/commit/dd0cae86dbe50dba43f772ae55d8faae0fc69a83', commit_message='Upload model', commit_description='', oid='dd0cae86dbe50dba43f772ae55d8faae0fc69a83', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs